In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm
from glob import glob
import tifffile as tif
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
import json
from sklearn.utils import shuffle
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *

In [ ]:
pip install -U albumentations

     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 37.6MB 80kB/s 
     |████████████████████████████████| 952kB 45.2MB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import cv2
from albumentations import HorizontalFlip, VerticalFlip, CenterCrop, Crop, Transpose, ChannelShuffle

In [ ]:
smooth = 1e-15

def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def focal_loss(y_true, y_pred):
    alpha=0.25
    gamma=2
    def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
        return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))
    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)
    return tf.reduce_mean(loss)

In [ ]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights='imagenet', input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = Concatenate()([outputs1, outputs2])

    model = Model(inputs, outputs)
    return model

In [ ]:
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"Error: creating directory with name {path}")

def read_data(x, y):
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    mask = cv2.imread(y, cv2.IMREAD_COLOR)
    return image, mask

def read_params():
    with open("params.json", "r") as f:
        data = f.read()
        params = json.loads(data)
        return params

def load_data(path):
    images_path = os.path.join(path, "image/*")
    masks_path  = os.path.join(path, "mask/*")

    images = glob(images_path)
    masks  = glob(masks_path)

    return images, masks

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_model_weight(path):
    with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef, 'bce_dice_loss': bce_dice_loss, 'focal_loss': focal_loss, 'iou': iou}):
        model = load_model(path)
    return model

In [ ]:
def augment_data(images, masks, save_path, augment=True):
    crop_size = (192-32, 256-32)
    size = (256, 192)

    for image, mask in tqdm(zip(images, masks), total=len(images)):
        image_name = image.split("/")[-1].split(".")[0]
        mask_name = mask.split("/")[-1].split(".")[0]

        x, y = read_data(image, mask)
        try:
            h, w, c = x.shape
        except Exception as e:
            image = image[:-1]
            x, y = read_data(image, mask)
            h, w, c = x.shape

        if augment == True:
            ## Transpose
            aug = Transpose(p=1)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            ## Channel Shuffle
            aug = ChannelShuffle(p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            images = [x, x2, x3]
            masks  = [y, y2, y3]
        else:
            images = [x]
            masks  = [y]

        idx = 0
        for i, m in zip(images, masks):
            i = cv2.resize(i, size)
            m = cv2.resize(m, size)

            tmp_image_name = f"{image_name}_{idx}.jpg"
            tmp_mask_name  = f"{mask_name}_{idx}.jpg"

            image_path = os.path.join(save_path, "image/", tmp_image_name)
            mask_path  = os.path.join(save_path, "mask/", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

def get_data(path, split=0.1):
    train_x = glob(os.path.join(path, "trainx/*"))
    train_y = glob(os.path.join(path, "trainy/*"))

    valid_x = glob(os.path.join(path, "validationx/*"))
    valid_y = glob(os.path.join(path, "validationy/*"))

    test_x = glob(os.path.join(path, "testx/*"))
    test_y = glob(os.path.join(path, "testy/*"))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
np.random.seed(42)
path = "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/skin_lesion_segmentation"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = get_data(path, split=0.1)

create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/train/image/")
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/train/mask/")
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/valid/image/")
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/valid/mask/")
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/test/image/")
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/test/mask/")

augment_data(train_x, train_y, "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/train/", augment=True)
augment_data(valid_x, valid_y, "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/valid/", augment=False)
augment_data(test_x, test_y, "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/test/", augment=False)

100%|██████████| 114/114 [01:15<00:00,  1.51it/s]


In [ ]:
def read_image(x):
    x = x.decode()
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    return image

def read_mask(y):
    y = y.decode()
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def parse_data(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = np.concatenate([y, y], axis=-1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([192, 256, 3])
    y.set_shape([192, 256, 2])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=32)
    dataset = dataset.map(map_func=parse_data)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch)
    return dataset

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files")

train_path = "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/train"
valid_path = "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/valid"

train_x = sorted(glob(os.path.join(train_path, "image", "*.jpg")))
train_y = sorted(glob(os.path.join(train_path, "mask", "*.jpg")))
train_x, train_y = shuffling(train_x, train_y)

valid_x = sorted(glob(os.path.join(valid_path, "image", "*.jpg")))
valid_y = sorted(glob(os.path.join(valid_path, "mask", "*.jpg")))

model_path = "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files/model.h5"
batch_size = 15
epochs = 10
lr = 1e-4
shape = (192, 256, 3)

model = build_model(shape)
metrics = [ "acc", dice_coef, iou, Recall(), Precision() ]

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

callbacks = [ ModelCheckpoint(model_path), ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20), CSVLogger("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files/data.csv"), TensorBoard(), EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False) ]

train_steps = (len(train_x)//batch_size)
valid_steps = (len(valid_x)//batch_size)

if len(train_x) % batch_size != 0:
    train_steps += 1

if len(valid_x) % batch_size != 0:
    valid_steps += 1

model.fit(train_dataset, epochs=epochs, validation_data=valid_dataset, steps_per_epoch=train_steps, validation_steps=valid_steps, callbacks=callbacks, shuffle=False)

80142336/80134624 [==============================] - 0s 0us/step
Epoch 1/10
37/37 [==============================] - 91s 1s/step - loss: 0.5686 - acc: 0.8369 - dice_coef: 0.4314 - iou: 0.2790 - recall: 0.7651 - precision: 0.4218 - val_loss: 0.5762 - val_acc: 0.1358 - val_dice_coef: 0.4238 - val_iou: 0.2694 - val_recall: 0.3093 - val_precision: 0.9682
Epoch 2/10
37/37 [==============================] - 36s 980ms/step - loss: 0.3787 - acc: 0.8240 - dice_coef: 0.6213 - iou: 0.4521 - recall: 0.9307 - precision: 0.7408 - val_loss: 0.4495 - val_acc: 0.2272 - val_dice_coef: 0.5505 - val_iou: 0.3804 - val_recall: 0.6719 - val_precision: 0.9605
Epoch 3/10
37/37 [==============================] - 37s 983ms/step - loss: 0.3405 - acc: 0.8749 - dice_coef: 0.6595 - iou: 0.4940 - recall: 0.9347 - precision: 0.8267 - val_loss: 0.4150 - val_acc: 0.9245 - val_dice_coef: 0.5850 - val_iou: 0.4142 - val_recall: 0.6586 - val_precision: 0.9754
Epoch 4/10
37/37 [==============================] - 37s 992ms/ste

In [ ]:
def read_image_(x):
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    return image

def read_mask_(y):
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask.astype(np.float32)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    return mask

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def parse(y_pred):
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = y_pred[..., -1]
    y_pred = y_pred.astype(np.float32)
    y_pred = np.expand_dims(y_pred, axis=-1)
    return y_pred

def evaluate_normal(model, x_data, y_data):
    THRESHOLD = 0.5
    total = []
    for i, (x, y) in tqdm(enumerate(zip(x_data, y_data)), total=len(x_data)):
        x = read_image_(x)
        y = read_mask_(y)
        _, h, w, _ = x.shape

        y_pred1 = parse(model.predict(x)[0][..., -2])
        y_pred2 = parse(model.predict(x)[0][..., -1])
        
        line = np.ones((h, 10, 3)) * 255.0
        
        all_images = [ x[0] * 255.0, line, mask_to_3d(y) * 255.0, line, mask_to_3d(y_pred1) * 255.0, line, mask_to_3d(y_pred2) * 255.0 ]
        mask = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files/results/{i}.png", mask)

smooth = 1.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
create_dir("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files/results/")

batch_size = 8

test_path = "/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/new_data/test"
test_x = sorted(glob(os.path.join(test_path, "image", "*.jpg")))
test_y = sorted(glob(os.path.join(test_path, "mask", "*.jpg")))
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

test_steps = (len(test_x)//batch_size)
if len(test_x) % batch_size != 0:
    test_steps += 1

model = load_model_weight("/content/drive/MyDrive/SEM6/DMW/DMW_Project/data/aesthetic/files/model.h5")

lr = 1e-4
metrics = [ "acc", dice_coef, iou, Recall(), Precision() ]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)
model.evaluate(test_dataset, steps=test_steps)
evaluate_normal(model, test_x, test_y)

15/15 [==============================] - 11s 234ms/step - loss: 0.2990 - acc: 0.9763 - dice_coef: 0.7010 - iou: 0.5425 - recall_1: 0.9283 - precision_1: 0.8023


100%|██████████| 114/114 [00:23<00:00,  4.88it/s]
